In [1]:
import geopandas as gpd
from odc.geo import XY, BoundingBox
from odc.geo.gridspec import GridSpec

In [2]:
# Create a gridspec
pacific_epsg = "EPSG:3832"
pacific_gridspec = GridSpec(crs=pacific_epsg, tile_shape=(3200,3200), resolution=30, origin=XY(120, -30))

In [3]:
# Select relevant tiles and convert to Geopandas
bounds = BoundingBox(120, -30, 280, 30, crs="EPSG:4326").to_crs(pacific_epsg)
geom = pacific_gridspec.geojson(bbox=bounds)

for feature in geom['features']:
    new_coords = []
    for coordinate in feature['geometry']['coordinates']:
        for coordinate_pair in coordinate:
            if coordinate_pair[0] < 0:
                coordinate_pair = (coordinate_pair[0] + 360, coordinate_pair[1])
            new_coords.append(coordinate_pair)
    feature['geometry']['coordinates'] = [new_coords]

tiles = gpd.GeoDataFrame.from_features(geom, crs="EPSG:4326")

In [4]:
# Get the Pacific admin boundaries
pacific_admin = gpd.read_file("pacific_admin.gpkg")

In [5]:
# Select tiles that intersect with the Pacific admin boundaries
# and copy the country name over
tiles_projected = tiles.to_crs(pacific_epsg)
admin_projected = pacific_admin.to_crs(pacific_epsg)

intersected = gpd.sjoin(tiles_projected, admin_projected, how="inner", predicate="intersects")

In [6]:
renamed = intersected[["geometry", "ISO2", "idx"]]
renamed = renamed.rename(columns={"idx": "tile_index", "ISO2": "country_code"})
renamed["bbox"] = renamed["geometry"].apply(lambda x: ",".join([str(b) for b in x.bounds]))
renamed = renamed[["bbox"]]

joined = tiles.join(renamed, how="inner", lsuffix="_left", rsuffix="_right")

In [7]:
joined.explore()

In [8]:
joined.to_file("grid_pacific.geojson")